In [ ]:
#%matplotlib notebook
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 9)

from matplotlib.colors import LogNorm
from matplotlib.ticker import FuncFormatter

import scipy
import scipy.stats
import math
import numpy as np
import pandas as pd

#import ipywidgets
#from ipywidgets import interact

import sys
sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline/")
sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline/utils/")
import common_functions as common

## Epsilon energy

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"

#WAVELET_LABEL = "WT-K-k-C1-m3-n4-s3"
WAVELET_LABEL = "WT-K-k-C1-m3-n4-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10"

PART = 0         # 0 for gamma, 1 for protons

FAINT_BRIGHT_BORDER = 100
LOW_CUT = 50
HIGH_CUT = 2000

full_df = pd.read_csv(CSV_FILE_PATH)

ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

In [ ]:
#full_df.groupby(["Type","Part","Success"]).count()
#full_df.groupby(["Type","Part"]).describe()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['Denergy_tc'], df['peSum_tc'], '.b', alpha=0.2,
        label="Tailcut ({})".format(len(df[df.peSum_tc > 0]['Denergy_tc'])))
ax.plot(df['Denergy_wt'], df['peSum_wt'], '.r', alpha=0.2,
        label="Wavelets ({})".format(len(df[df.peSum_wt > 0]['peSum_tc'])))

ax.axhline(y=LOW_CUT, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=HIGH_CUT, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel(r"$\mathcal{E}$ energy", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

ax.set_xlim(1e-4, 1e1)
ax.set_ylim(1e0, 1e5)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

title = r"$\mathcal{{E}}$ energy for {} and {} [{}]".format(TAILCUT_LABEL, WAVELET_LABEL, "Gamma" if PART == 0 else "Proton")
ax.set_title(title, fontsize=20)

In [ ]:
fig, ax = plt.subplots()

x1, y1 = df['Denergy_tc'], df['peSum_tc']
x2, y2 = df['Denergy_wt'], df['peSum_wt']

xbins = np.logspace(-3, 1, 30)
ybins = np.logspace(0, 5, 30)

counts1, xedges, yedges = np.histogram2d(x1, y1, bins=(xbins, ybins))
counts2, xedges, yedges = np.histogram2d(x2, y2, bins=(xbins, ybins))

max_value = max(np.max(counts1), np.max(counts2))

#levels = np.arange(10, 60, 10)
levels = np.array([0.01*max_value, 0.3*max_value, 0.5*max_value])

cs1 = plt.contour(xedges[:-1], yedges[:-1], counts1.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='blue', label="TC")
ax.clabel(cs1, inline=False, fontsize=12)

cs2 = plt.contour(xedges[:-1], yedges[:-1], counts2.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='red', label="WT")
ax.clabel(cs2, inline=False, fontsize=12)

ax.axhline(y=LOW_CUT, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=HIGH_CUT, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlim(1e-2, 1e1)
ax.set_ylim(1e0, 1e5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel(r"$\mathcal{E}$ energy", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

lines = [ cs1.collections[0], cs2.collections[0]]
labels = ["Tailcut ({})".format(len(df[df.peSum_tc > 0])), "Wavelets ({})".format(len(df[df.peSum_wt > 0]))]
ax.legend(lines, labels, prop={'size': 14}, loc='best', fancybox=True, framealpha=0.5)

title = r"$\mathcal{{E}}$ energy for {} and {} [{}]".format(TAILCUT_LABEL, WAVELET_LABEL, "Gamma" if PART == 0 else "Proton")
ax.set_title(title, fontsize=20)

In [ ]:
# %load /Users/jdecock/git/pub/jdhp/snippets/python/matplotlib/hist2d_scatter_plot_logscale_xy.py

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 9))

x1 = np.log10(df['Denergy_tc'])
y1 = np.log10(df['peSum_tc'])
x2 = np.log10(df['Denergy_wt'])
y2 = np.log10(df['peSum_wt'])

xbins1 = np.linspace(-4, 1, 100)
ybins1 = np.linspace(0, 6, 150)

xbins2 = np.linspace(-4, 1, 100)
ybins2 = np.linspace(0, 6, 150)

hist1, xedges1, yedges1 = np.histogram2d(x1, y1, bins=(xbins1, ybins1))
hist2, xedges2, yedges2 = np.histogram2d(x2, y2, bins=(xbins2, ybins2))

# See http://stackoverflow.com/questions/27156381/python-creating-a-2d-histogram-from-a-numpy-matrix

xidx1 = np.clip(np.digitize(x1, xedges1), 0, hist1.shape[0]-1)
yidx1 = np.clip(np.digitize(y1, yedges1), 0, hist1.shape[1]-1)
c1 = hist1[xidx1, yidx1]

xidx2 = np.clip(np.digitize(x2, xedges2), 0, hist2.shape[0]-1)
yidx2 = np.clip(np.digitize(y2, yedges2), 0, hist2.shape[1]-1)
c2 = hist2[xidx2, yidx2]

sc1 = ax1.scatter(x1, y1,
                  c=c1,
                  s=5,
                  marker='o',
                  #cmap='gnuplot2',
                  linewidth=0,
                  alpha=1)

sc2 = ax2.scatter(x2, y2,
                  c=c2,
                  s=5,
                  marker='o',
                  #cmap='gnuplot2',
                  linewidth=0,
                  alpha=1)

fig.colorbar(sc1, ax=ax1)
fig.colorbar(sc2, ax=ax2)

## Use "10^n" instead "n" as ticks label
#func_formatter = lambda x, pos: r'$10^{{{}}}$'.format(int(x))
#ax1.xaxis.set_major_formatter(FuncFormatter(func_formatter))
#ax2.xaxis.set_major_formatter(FuncFormatter(func_formatter))
#ax1.yaxis.set_major_formatter(FuncFormatter(func_formatter))
#ax2.yaxis.set_major_formatter(FuncFormatter(func_formatter))

ax1.axhline(y=np.log10(LOW_CUT), linewidth=2, linestyle='--', color='red', alpha=0.8)
ax1.axhline(y=np.log10(HIGH_CUT), linewidth=2, linestyle='--', color='red', alpha=0.8)
ax1.axhline(y=np.log10(FAINT_BRIGHT_BORDER), linewidth=2, linestyle=':', color='red', alpha=0.8)
ax2.axhline(y=np.log10(LOW_CUT), linewidth=2, linestyle='--', color='red', alpha=0.8)
ax2.axhline(y=np.log10(HIGH_CUT), linewidth=2, linestyle='--', color='red', alpha=0.8)
ax2.axhline(y=np.log10(FAINT_BRIGHT_BORDER), linewidth=2, linestyle=':', color='red', alpha=0.8)

ax1.set_xlabel(r"$\mathcal{E}$ energy", fontsize=20)
ax1.set_ylabel("npe", fontsize=20)
ax2.set_xlabel(r"$\mathcal{E}$ energy", fontsize=20)
ax2.set_ylabel("npe", fontsize=20)

#ax1.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)
#ax2.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

ax1.set_xlim(-4, 1)
ax1.set_ylim(0, 5)

ax2.set_xlim(-4, 1)
ax2.set_ylim(0, 5)

ax1.set_title(TAILCUT_LABEL, fontsize=20)
ax2.set_title(WAVELET_LABEL, fontsize=20)

title = r"$\mathcal{{E}}$ energy for {} and {} [{}]".format(TAILCUT_LABEL, WAVELET_LABEL, "Gamma" if PART == 0 else "Proton")
ax.set_title(title, fontsize=20)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

common.plot_hist2d(ax1, df['Denergy_tc'].dropna(how='any').values,
                   df['peSum_tc'].dropna(how='any').values,
                   "e_energy", "npe", logx=True, logy=True, logz=True)
common.plot_hist2d(ax2, df['Denergy_wt'].dropna(how='any').values,
                   df['peSum_wt'].dropna(how='any').values,
                   "e_energy", "npe", logx=True, logy=True, logz=True)

ax1.set_xlim(0.001, 1.)
ax2.set_xlim(0.001, 1.)

ax1.set_title("tc")
ax2.set_title("wt")